###Salary Prediction ###

In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [146]:
training_data = '../input/adult-training.csv'
test_data = '../input/adult-test.csv'
columns = ['Age','Workclass','fnlgwt','Education','Education Num','Marital Status',
           'Occupation','Relationship','Race','Sex','Capital Gain','Capital Loss',
           'Hours/Week','Country','Above/Below 50K']
train=pd.read_csv(training_data, names=columns)
test=pd.read_csv(test_data, names=columns)

##Data cleaning##

In [147]:
print(test.shape)
print(train.shape)

# deleted the firt rows of the both dataframes since these are errorneous
test.drop(test.index[0]).head()
train.drop(train.index[0]).head()

In [148]:
# a swiss knife tool for separating string columns to separate list and numerical to separate list
str_list=[]
for colname, colvalue in train.iteritems(): 
    if type(colvalue[1]) == str:
        str_list.append(colname) 
num_list = train.columns.difference(str_list)

In [149]:
# no null values its good
print(train.isnull().sum())

Looking at the unique values for the categorical columns, it seems there are is special charatcers ' ?'
inserted in the dataframe in place of null values we have to remove it and replace it wiyh nan

In [150]:
# the code will replace the special character to nan
for i in train.columns:
    train[i].replace(' ?', np.nan, inplace=True)
    

In [151]:
print(train.isnull().sum())

Picture is now clear. As you can see missing values in the WorkClass and Occupation are quite high. We will delete the missing value

In [152]:
train.dropna()

In [160]:
# defining the target variable
train['target']=train['Above/Below 50K'].apply(lambda x: 1 if x==' >50K' else 0)


In [ ]:
train.drop(['Above/Below 50K'], axis=1, inplace=True)

I dont believe that salary depends on the country the person belongs to. Hence deleting the country and old target variable

In [157]:
train.drop(['Country'], axis=1, inplace=True)
# assigning the target to Y variable
Y=train['target']

In [158]:
# since target is already assigned I Will drop the target from the train
train.drop(['target'], axis=1, inplace=True)

I will now create dummies for the categorical variables

In [ ]:
str_list=['Workclass',
 'Education',
 'Marital Status',
 'Occupation',
 'Relationship',
 'Race',
 'Sex']

train_set=pd.get_dummies(train, columns=str_list)


In [95]:
#creating array fro features
X=train_set.values
X

Variance Threshold is a univariate  approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples.
As an example, suppose that we have a dataset with boolean features, and we want to remove all features that are either one or zero (on or off) in more than 80% of the samples. Boolean features are Bernoulli random variables, and the variance of such variables is given by
The below approach removes variable which have more than 80% values are either 0 or 1

In [ ]:
from sklearn.feature_selection import VarianceThreshold
def variance_threshold_select(df, thresh=0.0, na_replacement=-999):
    df1 = df.copy(deep=True) # Make a deep copy of the dataframe
    selector = VarianceThreshold(thresh)
    selector.fit(df1.fillna(na_replacement)) # Fill NA values as VarianceThreshold cannot deal with those
    df2 = df.loc[:,selector.get_support(indices=False)] # Get new dataframe with columns deleted that have NA values
    return df2

In [ ]:
df2=variance_threshold_select(train_set, thresh=(.9 * (1 - .9)))

In [ ]:
col=df2.columns
X=df2.values

In [ ]:
len(col)

Modelling Process

In [138]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [139]:
model = LogisticRegression()
rfe = RFE(model, 7)
fit = rfe.fit(X, Y)

In [1]:
logistic = model.fit(X, Y)
ypred_train = logistic.predict(X)

In[38]: #checking the classification accuracy
accuracy_score(Y,ypred_train)

In [144]:
pd.DataFrame(list(zip(fit.ranking_, col)))


In [141]:

fit.ranking_

In [142]:
fit.support_